In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import xgboost as xgb

/home/sgu/miniconda2/envs/GAPM/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
df_properties_2016 = pd.read_csv("properties_2016.csv")
df_train_2016 = pd.read_csv("train_2016_v2.csv")
df_properties_2016.shape, df_train_2016.shape

((2985217, 58), (90275, 3))

In [6]:
df = pd.merge?

In [7]:
df = df_train_2016.merge(df_properties_2016, on='parcelid')
df.shape

(90275, 60)

In [15]:
zip(np.percentile(np.abs(df.logerror.values),range(0,101,10)),range(0,101,10))

[(0.0, 0),
 (0.0050000000000000001, 10),
 (0.0109, 20),
 (0.016899999999999998, 30),
 (0.024299999999999999, 40),
 (0.032500000000000001, 50),
 (0.044000000000000004, 60),
 (0.058299999999999991, 70),
 (0.0843, 80),
 (0.14499999999999999, 90),
 (4.7370000000000001, 100)]

In [17]:
sel = np.random.rand(len(df))<0.8
df_train, df_test = df[sel], df[~sel]
df_train.shape, df_test.shape

((72257, 60), (18018, 60))

In [39]:
df_x = df.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'],axis=1)
df_y = df['logerror']
df_x.shape, df_y.shape

((90275, 55), (90275,))

In [40]:
sel = df_x.dtypes == object
cols_obj = df_x.dtypes[sel].index.values
df_x[cols_obj].apply(lambda s: dict(s.value_counts()), axis=0)

hashottuborspa        {True: 2365}
fireplaceflag          {True: 222}
taxdelinquencyflag    {u'Y': 1783}
dtype: object

In [41]:
for c in cols_obj: 
    df_x[c] = (df_x[c] == True) | (df_x[c]=='Y')

In [42]:
df_x.dtypes.value_counts()

float64    52
bool        3
dtype: int64

In [43]:
sel = np.random.rand(len(df_x))<0.8
df_x_train, df_x_test = df_x[sel], df_x[~sel]
df_y_train, df_y_test = df_y[sel], df_y[~sel]
df_x_train.shape, df_y_train.shape, df_x_test.shape, df_y_test.shape

((72295, 55), (72295,), (17980, 55), (17980,))

In [48]:
d_train = xgb.DMatrix(df_x_train, label=df_y_train.values)
d_test = xgb.DMatrix(df_x_test, label=df_y_test.values)
d_train.num_row(), d_train.num_col(), d_test.num_row(), d_test.num_col()

(72295L, 55L, 17980L, 55L)